In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import stock_module as sm
import random
import os
import datetime
import time

In [2]:
stock_name = "EGSER"
start_date = "01-01-2014"
end_date = "31-05-2020"
RSI_lower_bound = 30
RSI_upper_bound = 65
RSI_reference_day_count = 10

In [3]:
all_stock_value = sm.values_from_database()

In [4]:
frame = sm.all_movingAverages(all_stock_value, stock_name, [5,20,50], start_date, end_date)
frame["change"] = frame["MA_5"] - frame["MA_50"] 
frame["new"] = frame["change"].shift(1)
frame = frame.dropna()
frame["test"] = frame["change"] * frame["new"]
frame = frame.loc[frame["test"]<0]
frame["status"] = frame["change"] * frame["test"]

frame['suggestion'] = np.where(frame['status']<0, 'buy', 'sell')

del frame["new"]
del frame["test"]
del frame["status"]
del frame["change"]

frame["date"] = frame.index

values = sm.values_from_database()
values = values[[str(stock_name)]]
values["date"] = values.index
values = values.dropna()

frame = pd.merge(frame, values, on="date", how="left")

RSI = sm.RSI_indicator(str(stock_name), RSI_reference_day_count, start_date, end_date)

RSI["date"] = RSI.index

del RSI["HGDG_KAPANIS"]
del RSI["Change"]
del RSI["Gain"]
del RSI["Loss"]
del RSI["RS"]

frame = pd.merge(frame, RSI, on="date", how="left")

frame["test"] = frame["MA_5"] - frame["MA_20"]
frame['suggestion2'] = np.where(frame['test']>0, 'buy', 'sell')
del frame["test"]
final = frame.loc[(frame["RSI"]>RSI_upper_bound) | (frame["RSI"]<RSI_lower_bound)]

final_buy = final.loc[(final["suggestion"]=="buy") & (final["suggestion2"]=="buy")]
final_sell = final.loc[(final["suggestion"]=="sell") & (final["suggestion2"]=="sell")]
finall = pd.concat([final_buy, final_sell])
finall = finall.sort_values(by=["date"])

/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg


In [5]:
finall

,MA_5,MA_20,MA_50,suggestion,date,EGSER,RSI,suggestion2
0,1.93066,1.850025,1.926954,buy,2014-02-17,1.9605,81.237174,buy
3,2.62258,2.697865,2.660228,sell,2014-10-16,2.5734,17.501627,sell
7,3.14802,3.208715,3.162318,sell,2015-03-04,3.0343,21.567126,sell
8,3.17416,3.086185,3.159702,buy,2015-03-31,3.2187,71.122685,buy
16,2.72026,2.659225,2.700950,buy,2015-10-09,2.8044,82.741329,buy
17,3.23168,3.409410,3.267286,sell,2015-12-16,3.2416,27.277737,sell
23,3.56818,3.781810,3.635486,sell,2016-05-03,3.2910,13.262403,sell
27,3.33394,3.383010,3.401896,sell,2016-08-16,3.1838,24.040528,sell
34,2.80274,2.701695,2.800762,buy,2017-04-07,2.8456,72.719758,buy
36,3.79166,3.614420,3.775028,buy,2017-11-02,3.9731,80.263158,buy


---

In [6]:
def buy_sell(stock_name, start_date, end_date, RSI_lower_bound, RSI_upper_bound, RSI_reference_day_count):
    frame = sm.all_movingAverages(all_stock_value, stock_name, [5,20,50], start_date, end_date)
    frame["change"] = frame["MA_5"] - frame["MA_50"] 
    frame["new"] = frame["change"].shift(1)
    frame = frame.dropna()
    frame["test"] = frame["change"] * frame["new"]
    frame = frame.loc[frame["test"]<0]
    frame["status"] = frame["change"] * frame["test"]

    frame['suggestion'] = np.where(frame['status']<0, 'buy', 'sell')

    del frame["new"]
    del frame["test"]
    del frame["status"]
    del frame["change"]

    frame["date"] = frame.index

    values = sm.values_from_database()
    values = values[[str(stock_name)]]
    values["date"] = values.index
    values = values.dropna()

    frame = pd.merge(frame, values, on="date", how="left")

    RSI = sm.RSI_indicator(str(stock_name), RSI_reference_day_count, start_date, end_date)

    RSI["date"] = RSI.index

    del RSI["HGDG_KAPANIS"]
    del RSI["Change"]
    del RSI["Gain"]
    del RSI["Loss"]
    del RSI["RS"]

    frame = pd.merge(frame, RSI, on="date", how="left")

    frame["test"] = frame["MA_5"] - frame["MA_20"]
    frame['suggestion2'] = np.where(frame['test']>0, 'buy', 'sell')
    del frame["test"]
    final = frame.loc[(frame["RSI"]>RSI_upper_bound) | (frame["RSI"]<RSI_lower_bound)]

    final_buy = final.loc[(final["suggestion"]=="buy") & (final["suggestion2"]=="buy")]
    final_sell = final.loc[(final["suggestion"]=="sell") & (final["suggestion2"]=="sell")]
    finall = pd.concat([final_buy, final_sell])
    finall = finall.sort_values(by=["date"])
    
    finall["stock_name"] = str(stock_name)
    finall["value"] = finall.iloc[:,5]
    finall.drop(finall.columns[5], axis = 1, inplace = True) 
    
    finall = finall[["date", "stock_name", "MA_5", "MA_20", "MA_50", "value", "RSI", "suggestion", "suggestion2"]]
    
    return finall

In [14]:
def buy_sell(stock_name, start_date, end_date, RSI_lower_bound, RSI_upper_bound, RSI_reference_day_count):
    frame = sm.all_movingAverages(all_stock_value, stock_name, [5,20,50], start_date, end_date)
    frame["change"] = frame["MA_5"] - frame["MA_50"] 
    frame["new"] = frame["change"].shift(1)
    frame = frame.dropna()
    frame["test"] = frame["change"] * frame["new"]
    frame = frame.loc[frame["test"]<0]
    frame["status"] = frame["change"] * frame["test"]

    frame['suggestion'] = np.where(frame['status']<0, 'buy', 'sell')

    del frame["new"]
    del frame["test"]
    del frame["status"]
    del frame["change"]

    frame["date"] = frame.index

    values = sm.values_from_database()
    values = values[[str(stock_name)]]
    values["date"] = values.index
    values = values.dropna()

    frame = pd.merge(frame, values, on="date", how="left")

    RSI = sm.RSI_indicator(str(stock_name), RSI_reference_day_count, start_date, end_date)

    RSI["date"] = RSI.index

    del RSI["HGDG_KAPANIS"]
    del RSI["Change"]
    del RSI["Gain"]
    del RSI["Loss"]
    del RSI["RS"]

    frame = pd.merge(frame, RSI, on="date", how="left")

    frame["test"] = frame["MA_5"] - frame["MA_20"]
    frame['suggestion2'] = np.where(frame['test']>0, 'buy', 'sell')
    del frame["test"]
    final = frame.loc[(frame["RSI"]>RSI_upper_bound) | (frame["RSI"]<RSI_lower_bound)]

    final_buy = final.loc[(final["suggestion"]=="buy") & (final["suggestion2"]=="buy")]
    final_sell = final.loc[(final["suggestion"]=="sell") & (final["suggestion2"]=="sell")]
    finall = pd.concat([final_buy, final_sell])
    finall = finall.sort_values(by=["date"])
    
    finall["stock_name"] = str(stock_name)
    finall["value"] = finall.iloc[:,5]
    finall.drop(finall.columns[5], axis = 1, inplace = True) 
    
    finall = finall[["date", "stock_name", "MA_5", "MA_20", "MA_50", "value", "RSI", "suggestion", "suggestion2"]]
    
    return finall

In [15]:
test = buy_sell("ARDYZ", "01-01-2014", "31-05-2020", 30, 65, 10)

/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg


In [16]:
test

,MA_5,MA_20,MA_50,suggestion,ARDYZ,date,suggestion2,stock_name,value


In [25]:
def buy_sell_frame(stock_names, start_date, end_date, RSI_lower_bound, RSI_upper_bound, RSI_reference_day_count):
    
    frames = []
    # stock_names = list(sm.values_from_database())
    for stock_name in stock_names:
        try:
            frame = buy_sell(str(stock_name), start_date, end_date, RSI_lower_bound, RSI_upper_bound, RSI_reference_day_count)
            frames.append(frame)
        except:
            print(stock_name)
        
    frames = pd.concat(frames)
    
    return frames      

In [30]:
stock_names = list(sm.values_from_database())

In [ ]:
final_test = buy_sell_frame(stock_names, "01-01-2014", "31-05-2020", 30, 65, 10)

/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: d

/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: di

/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: 

/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: d

/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: in

/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: divide by zero encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: invalid value encountered in double_scalars
  RS = gainsAvg/lossesAvg
/Users/mturan/Desktop/Scripts/stock_analysis/stock_module.py:863: RuntimeWarning: di

In [ ]:
final_test

In [ ]:
final_test.to_pickle("buy_sell_frame")

,date,stock_name,MA_5,MA_20,MA_50,value,RSI,suggestion,suggestion2
0,2014-01-03,ACSEL,1.57770,1.559690,1.571896,1.6551,0.000000,buy,buy
2,2014-03-26,ACSEL,1.58522,1.557555,1.583720,1.6094,83.385093,buy,buy
3,2014-04-28,ACSEL,1.59972,1.649965,1.602038,1.5745,8.677686,sell,sell
4,2014-06-18,ACSEL,1.60510,1.580120,1.599994,1.6121,73.325062,buy,buy
6,2014-09-11,ACSEL,1.57718,1.539275,1.576298,1.5772,77.591036,buy,buy
...,...,...,...,...,...,...,...,...,...
31,2018-05-23,ADBGR,3.22744,3.291720,3.266152,3.1017,13.972056,sell,sell
32,2018-07-02,ADBGR,3.20050,3.115945,3.196960,3.2883,79.848594,buy,buy
38,2018-11-29,ADBGR,3.14236,3.080350,3.138896,3.1785,66.677556,buy,buy
40,2019-07-01,ADBGR,3.66974,3.596370,3.643848,3.7671,65.794769,buy,buy
